# Agents

This notebook will go through how Agents are defined in `autogen`, and walk through the subclasses of `Agent` work with code examples.

In [ ]:
# Import necessary libraries
import os
import json
import chess
import openai
import autogen
import tempfile
import chromadb
from pathlib import Path
from collections import defaultdict
from typing import Any, Dict, List, Optional, Union

from utils.api_utils import load_config_list_from_dotenv

config_list = load_config_list_from_dotenv()

## agentchat.agent

The Agent class is an abstract base class designed to represent an AI agent that can communicate with other agents and perform actions. It provides a high-level interface for sending, receiving, and processing messages between agents. Agents can differ in how they handle received messages and what actions they perform in the receive method.

**Extending the Agent class:**
To extend the `Agent` class, one might create a subclass and implement the abstract methods: `send`, `receive`, `reset`, and `generate_reply`. The subclass might have additional methods and attributes depending on the specific functionalities needed.

Example of extending the Agent class based on the conversation and documentation:

In [1]:
from autogen.agentchat import Agent
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union


class MyAgent(Agent):
    def __init__(self, name: str, some_config: Dict):
        super().__init__(name)
        self.config = some_config  # Additional configuration if needed

    def send(self, message: Union[Dict, str], recipient: "Agent", request_reply: Optional[bool] = None):
        # Implementation of the send method to send a message to another agent
        pass

    async def a_send(self, message: Union[Dict, str], recipient: "Agent", request_reply: Optional[bool] = None):
        # Implementation of the async send method to send a message to another agent
        pass

    def receive(self, message: Union[Dict, str],
                sender: "Agent", request_reply: Optional[bool] = None):
        # Implementation of the receive method to process received messages from another agent
        pass

    async def a_receive(self, message: Union[Dict, str],
                        sender: "Agent", request_reply: Optional[bool] = None):
        # Implementation of the async receive method to process received messages from another agent
        pass

    def reset(self):
        # Reset the state of the agent
        pass

    def generate_reply(self, messages: Optional[List[Dict]] = None,
                       sender: Optional["Agent"] = None, **kwargs) -> Union[str, Dict, None]:
        # Generate a reply based on the received messages
        pass

    async def a_generate_reply(self, messages: Optional[List[Dict]] = None,
                               sender: Optional["Agent"] = None, **kwargs) -> Union[str, Dict, None]:
        # Async method to generate a reply based on the received messages
        pass

## agentchat.ConversableAgent

**Purpose**: A class for creating generic conversable agents that can be configured as an assistant or user proxy. In simpler terms, this class is enhancing the basic `Agent` by adding conversational capabilities, managing conversations and messages, and allowing customization and control over responses and actions in the conversation. It provides properties and methods to manage system messages, reply functions, chat messages, and to perform actions like sending messages to other agents.

- When a conversation is initiated, `ConversableAgent` starts by sending an initial message and prepares to handle subsequent messages.
- For each received message, it follows a series of checks and executes corresponding actions like generating replies, executing codes, or making function calls.
- It can involve a human in the conversation when needed, asking for inputs or decisions.
- It maintains flexibility and extensibility by allowing customization of its behaviors and dynamic addition of new capabilities.


**Key Methods:**
- `register_reply`: Registers a reply function based on a specified trigger.
- `send`: Sends a message to another agent.
- `receive`: Receives a message from another agent and sends a reply or stops.
- `initiate_chat`: Initiates a chat with the recipient agent, resetting the auto reply counter and clearing chat history if specified.
- `reset`: Resets the agent, clearing all registered reply functions and the chat history.
- `generate_reply`: Generates a reply based on the conversation history and the sender using registered auto reply functions.


**Example Workflow:**
1. Create instances of AssistantAgent and ConversableAgent.
2. Register reply functions to ConversableAgent using register_reply method.
3. Use send method to send messages between agents.
4.  Use receive method to receive messages and automatically generate replies based on registered reply functions.
5. Reset agents and clear chat history when needed using reset and clear_history methods respectively.

## agentchat.AssistantAgent

**Purpose:** `AssistantAgent` is a specialized subclass of `ConversableAgent`, designed primarily to act as a task-solving assistant using Language Learning Models (LLMs). It is programmed to suggest Python code blocks or shell scripts for the user to execute, aiming to solve tasks step by step while providing clear plans and explanations. It emphasizes providing structured, task-oriented solutions but does not execute the code by default, leaving the execution to the user.

- It starts with a default system message which guides the assistant on how to approach and solve tasks and how to communicate the solutions to the user.
- It can generate replies based on LLMs and can maintain a conversation around solving specific tasks, providing coding suggestions and debugging help.
- It maintains a structured and task-focused approach, emphasizing clarity, verification, and providing evidence for the solutions given.


**Key Methods:**

`__init__`: Initializes the `AssistantAgent` with specific configurations including LLM configuration and system message.
Inherits methods from `ConversableAgent` like `register_reply`, `send`, `receive`, `initiate_chat`, `reset`, and `generate_reply`.


## agentchat.UserProxyAgent

**Purpose**: UserProxyAgent is another subclass of ConversableAgent that acts as a proxy for the user, designed to execute code and provide user-like feedback to other agents. It is more interactive and flexible, emphasizing obtaining human inputs and allowing modifications to the way it gets human input and executes code. By default, it prompts for human input every time a message is received and has code execution enabled, allowing for a more dynamic and responsive conversation flow.

- It can execute code by default and is more focused on interactivity, acting as a user representative in conversations.
- It is highly customizable, allowing methods to be overridden to modify how it gets human input, executes code, and generates initial messages.
- It maintains the capability to register replies and can be configured to auto-reply based on specific conditions or triggers.


**Key Methods:**

- `human_input_mode (Optional[str])`: Strategy to determine when to ask for human inputs. It has three possible values:
    - "ALWAYS": Prompts for human input every time a message is received.
    - "TERMINATE": Only prompts for human input when a termination message is received or the number of auto replies reaches max_consecutive_auto_reply.
    - "NEVER": Never prompts for human input.

**Workflow:**
1. Initialization: Create an instance of `UserProxyAgent` with the desired configuration.
2. Integration: Integrate the `UserProxyAgent` within a chat system or multi-agent environment.
3. Interaction: The agent can receive and process messages, execute code, provide feedback to other agents, and prompt for human input based on its configuration.
